In [ ]:
# TASK 2 - IMAGE CLASSIFICATION
# CODTECH Internship - Patel Jiii

import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import sys, os

sys.path.append("../src")
from utils import ensure_dir, plot_training, save_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DATA_DIR = "../data/images"
MODEL_DIR = "../models"
ensure_dir(MODEL_DIR)


In [ ]:
transform = {
    "train": transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ]),
    "val": transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor()
    ])
}

train_ds = datasets.ImageFolder(f"{DATA_DIR}/train", transform=transform["train"])
val_ds = datasets.ImageFolder(f"{DATA_DIR}/val", transform=transform["val"])

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16)


In [ ]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(train_ds.classes))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

history = {"train_loss": [], "val_loss": [], "val_acc": []}


In [ ]:
EPOCHS = 3
best_acc = 0

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0

    for img, lbl in train_loader:
        img, lbl = img.to(device), lbl.to(device)
        optimizer.zero_grad()
        out = model(img)
        loss = criterion(out, lbl)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    model.eval()
    val_loss, correct = 0, 0

    with torch.no_grad():
        for img, lbl in val_loader:
            img, lbl = img.to(device), lbl.to(device)
            out = model(img)
            loss = criterion(out, lbl)
            val_loss += loss.item()
            correct += (out.argmax(1) == lbl).sum().item()

    val_acc = correct / len(val_ds)

    history["train_loss"].append(running_loss / len(train_loader))
    history["val_loss"].append(val_loss / len(val_loader))
    history["val_acc"].append(val_acc)

    print(f"Epoch {epoch+1}/{EPOCHS} | Val Acc: {val_acc:.3f}")

    if val_acc > best_acc:
        save_model(model, f"{MODEL_DIR}/saved_model.pth")
        best_acc = val_acc

plot_training(history, MODEL_DIR)
